In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D,Reshape, LSTM, Dropout, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.utils import shuffle

bases={'A':np.array([0,0,0,1]), 'C':np.array([0,0,1,0]), 'G':np.array([0,1,0,0]), 'T':np.array([1,0,0,0])}

def Kmers_funct(seq, size):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

def one_hot_encode_2(y,num_classes):
    y_encoded=[]
    for value in y:
	    letter = [0 for _ in range(num_classes)]
	    letter[value] = 1
	    y_encoded.append(letter)
    return np.array(y_encoded,dtype=np.float16)



In [2]:
import random

def selectref(dicref,el):
    return random.choice(dicref[el])

dicref={'U':['T'],
'R':['A','G'],
'Y':['C','T'],
'S':['G','C'],
'W':['A','T'],
'K':['G','T'],
'M':['A','C'],
'B':['C','G','T'],
'D':['A','G','T'],
'H':['A','C','T'],
'V':['A','C','G'],
'N':['A','T','G','C'],
'A':['A'],
'C':['C'],
'G':['G'],
'T':['T']}

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!pip install BioPython

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from Bio import SeqIO
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

unique_elems=dict()

all_sequences=[]

for sequence in SeqIO.parse('/content/drive/My Drive/ncbi_16s_18s_merged.fasta', "fasta"):
    unique_elems[sequence.description.split()[1]]=0
    seq=""
    for el in sequence.seq:
      seq += (selectref(dicref,el))
    
    words = Kmers_funct(seq, size=6)
    joined_sentence = ' '.join(words)
    all_sequences.append(joined_sentence)

X=cv.fit_transform(all_sequences).toarray()

print(X.shape)



    
    

    

(24855, 4096)


In [5]:
i=0
x_data=[]
y_data=[]
for el in unique_elems.keys():
    unique_elems[el]=i
    i+=1

i=0
for sequence in SeqIO.parse('/content/drive/My Drive/ncbi_16s_18s_merged.fasta', "fasta"):
    x_data.append(X[i])
    y_data.append(unique_elems[sequence.description.split()[1]])
    i+=1

lm=len(unique_elems)

class_count=dict()

for i in range(lm):
  class_count[i]=0

for el in y_data:
  class_count[el]+=1



In [6]:


x_train=[]
y_train=[]

rare_items=[]

for item in class_count.keys():
  if class_count[item] < 2:
    rare_items.append(item)

i=0
for el in y_data:
  if el in rare_items:
    x_train.append(x_data[i])
    y_train.append(el)

  i+=1


x_data,y_data=shuffle(x_data,y_data)

x_train+=x_data[:int(len(x_data)*0.8)]
y_train+=y_data[:int(len(y_data)*0.8)]


x_test=x_data[int(len(x_data)*0.8):]
y_test=y_data[int(len(y_data)*0.8):]

print(len(x_train))

y_train=one_hot_encode_2(y_train,len(unique_elems))
y_test=one_hot_encode_2(y_test,len(unique_elems))

x_train=np.array(x_train,dtype=np.float32)
x_train=np.reshape(x_train,(-1,4096,1))

x_test=np.array(x_test,dtype=np.float32)
x_test=np.reshape(x_test,(-1,4096,1))

21873


In [7]:
import os
checkpoint_path = "/content/drive/My Drive/training_genomics_kmer/{epoch:03d}/cp-{epoch:04d}"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [8]:
model=Sequential()
model.add((Conv1D(filters=6, kernel_size=6, activation='relu')))
model.add((Conv1D(filters=3, kernel_size=6, activation='relu')))
model.add(Flatten())
model.add(Dense(units=1024,activation='relu'))
model.add(Dense(units=lm, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights("/content/drive/My Drive/training_genomics_kmer/010/cp-0010")
early_stopping = EarlyStopping(monitor='loss', patience=2, mode='min')

history = model.fit(x_train[0:1], y_train[0:1], epochs=1, verbose=1)
model.summary()
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data= (x_test,y_test),callbacks=[cp_callback])


1/1 [==============================] - 7s 7s/step - loss: 2.7569e-04 - accuracy: 1.0000
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 4091, 6)           42        
                                                                 
 conv1d_1 (Conv1D)           (None, 4086, 3)           111       
                                                                 
 flatten (Flatten)           (None, 12258)             0         
                                                                 
 dense (Dense)               (None, 1024)              12553216  
                                                                 
 dense_1 (Dense)             (None, 4283)              4390075   
                                                                 
Total params: 16,943,444
Trainable params: 16,943,444
Non-trainable params: 0
______________________

In [9]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [9]:
from sklearn import metrics

preds=model.predict(x_test)

y_pred=[np.argmax(el) for el in preds]
y_true=[np.argmax(el) for el in y_test]

print(metrics.confusion_matrix(y_true, y_pred))

print(metrics.classification_report(y_true, y_pred, digits=3))


[[ 3  0  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  0 16 ...  0  0  0]
 ...
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  1]]
              precision    recall  f1-score   support

           0      1.000     1.000     1.000         3
           1      1.000     1.000     1.000         1
           2      1.000     1.000     1.000        16
           3      1.000     1.000     1.000         4
           4      1.000     1.000     1.000         1
           5      1.000     1.000     1.000         3
           7      1.000     1.000     1.000         1
           8      1.000     1.000     1.000         1
          10      1.000     1.000     1.000         2
          12      1.000     1.000     1.000         9
          14      1.000     1.000     1.000         7
          16      1.000     1.000     1.000         1
          19      1.000     1.000     1.000         1
          20      1.000     1.000     1.000         2
          21      1.000     1.000

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1